### Display live camera feed

So let's get started.  First, let's initialize and display our camera like we did in the *teleoperation* notebook.  

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task).
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.
###ライブカメラフィードを表示する

それでは始めましょう。 まず、* teleoperation *ノートブックで行ったようにカメラを初期化して表示しましょう。

>ニューラルネットワークは、入力として224x224ピクセルの画像を受け取ります。 カメラをそのサイズに設定して、データセットのファイルサイズを最小化します（このタスクで機能することをテストしました）。
>シナリオによっては、より大きな画像サイズでデータを収集し、後で目的のサイズにダウンスケールする方が良い場合があります。

Awesome, next let's create a few directories where we'll store all our data.  We'll create a folder ``dataset`` that will contain two sub-folders ``free`` and ``blocked``, 
where we'll place the images for each scenario.
素晴らしい、次にすべてのデータを保存するいくつかのディレクトリを作成しましょう。 各シナリオの画像を配置する2つのサブフォルダーを無料でブロックするフォルダーデータセットを作成します。

In [2]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')

If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about as many ``free`` images as ``blocked`` images.  It also helps to know how many images we've collected overall.
左側のJupyterファイルブラウザーを更新すると、これらのディレクトリが表示されます。 次に、スナップショットを保存するために使用するいくつかのボタンを作成して表示します
クラスラベルごと。 また、これまでに収集した各カテゴリの画像の数を表示するテキストボックスを追加します。 これは便利です
必ず「ブロックされた」画像と同じ数の「無料の」画像を収集してください。 また、全体で収集した画像の数を知るのにも役立ちます。

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.
左側のJupyterファイルブラウザーを更新すると、これらのディレクトリが表示されます。 次に、各クラスラベルのスナップショットを保存するために使用するいくつかのボタンを作成して表示します。 また、これまでに収集した各カテゴリの画像の数を表 示するテキストボックスを追加します。 これは、ブロックされた画像と同じ数の無料画像を確実に収集したいので便利です。 また、全体で収集した画像の数を知るのにも役立ちます。

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Great! Now the buttons above should save images to the ``free`` and ``blocked`` directories.  You can use the Jupyter Lab file browser to view these files!

Now go ahead and collect some data 

1. Place the robot in a scenario where it's blocked and press ``add blocked``
2. Place the robot in a scenario where it's free and press ``add free``
3. Repeat 1, 2

> REMINDER: You can move the widgets to new windows by right clicking the cell and clicking ``Create New View for Output``.  Or, you can just re-display them
> together as we will below

Here are some tips for labeling data

1. Try different orientations
2. Try different lighting
3. Try varied object / collision types; walls, ledges, objects
4. Try different textured floors / objects;  patterned, smooth, glass, etc.

Ultimately, the more data we have of scenarios the robot will encounter in the real world, the better our collision avoidance behavior will be.  It's important
to get *varied* data (as described by the above tips) and not just a lot of data, but you'll probably need at least 100 images of each class (that's not a science, just a helpful tip here).  But don't worry, it goes pretty fast once you get going :)

すばらしいです！これで、上のボタンは画像を `` free``および `` blocked``ディレクトリに保存するはずです。 Jupyter Labファイルブラウザを使用して、これらのファイルを表示できます！

さあ、データを集めましょう

1.ロボットがブロックされているシナリオにロボットを置き、「ブロックされた追加」を押します
2.ロボットを空いているシナリオに置き、「add free」を押します
3. 1、2を繰り返します

>リマインダー：セルを右クリックして「出力用の新しいビューを作成」をクリックすると、ウィジェットを新しいウィンドウに移動できます。または、それらを再表示することもできます
>以下で一緒に

データにラベルを付けるためのヒントを次に示します

1.別の向きを試してください
2.別の照明を試す
3.さまざまなオブジェクト/衝突タイプを試してください。壁、棚、オブジェクト
4.さまざまなテクスチャの床/オブジェクトを試してください。パターン、滑らか、ガラスなど

最終的に、ロボットが現実の世界で遭遇するシナリオのデータが多いほど、衝突回避の挙動は良くなります。それは重要です
（上記のヒントで説明した）*さまざまな*データを取得するには、大量のデータだけでなく、おそらく各クラスの少なくとも100個の画像が必要です（これは科学ではなく、ここで役立つヒントです）。しかし、心配しないでください、あなたが始めたらかな り速くなります:)

In [5]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Again, let's close the camera conneciton properly so that we can use the camera in the later notebook.

In [ ]:
camera.stop()

## Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output
＃＃ 次

十分なデータを収集したら、トレーニングのためにそのデータをGPUデスクトップまたはクラウドマシンにコピーする必要があります。 まず、次の* terminal *コマンドを呼び出して圧縮します
データセットフォルダを1つの* zip *ファイルにまとめます。

>！ prefixは、セルを* shell *（または* terminal *）コマンドとして実行することを示します。

>以下のzipコマンドの-rフラグは* recursive *を示すため、すべてのネストされたファイルが含まれます。-qフラグは* quiet *を示すため、zipコマンドは出力を出力しません。

In [6]:
!zip -r -q dataset.zip dataset

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model.ipynb``.

So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!
Jupyter Labファイルブラウザーに `` dataset.zip``という名前のファイルが表示されます。 Jupyter Labファイルブラウザを使用して、右クリックして「ダウンロード」を選択し、zipファイルをダウンロードする必要があります。

次に、このデータをGPUデスクトップまたはクラウドマシン（これを*ホスト*と呼びます）にアップロードして、衝突回避ニューラルネットワークをトレーニングする必要があります。 トレーニングをセットアップしたと仮定します
JetBot WiKiで説明されているマシン。 お持ちの場合は、「http：// <host_ip_address>：8888」に移動して、ホストで実行されているJupyter Lab環境を開くことができます。 そこで開く必要があるノートブックは、「collision_avoidance / train_model.ipynb」と呼ばれます。

だからあなたのトレーニングマシンに向かい、そこの指示に従ってください！ モデルがトレーニングされると、ロボットJupyter Labの環境に戻り、モデルをライブデモに使用します！